In [44]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os as os
from citipy import citipy
# Import the API key.
from config import weather_api_key
# Import the requests library.
import requests
# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

In [45]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=100)
lngs = np.random.uniform(low=-180.000, high=180.000, size=100)
lat_lngs = zip(lats, lngs)
lat_lngs

In [46]:
coordinates = list(lat_lngs)

In [47]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

81

In [48]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}*{city_url }")
    # Add 1 to the record count.
    record_count += 1
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # print(city_weather)
        # Parse out the needed data.
       
        city_lat = f'{city_weather["coord"]["lat"]:.2f}'
        city_lng = f'{city_weather["coord"]["lon"]:.2f}'
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_current_description = city_weather["weather"][0]["main"] 
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description":city_current_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | khani*http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=7735c52e8cd85cbee274b995f12a83a3&q=khani
Processing Record 2 of Set 1 | qaanaaq*http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=7735c52e8cd85cbee274b995f12a83a3&q=qaanaaq
Processing Record 3 of Set 1 | desaguadero*http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=7735c52e8cd85cbee274b995f12a83a3&q=desaguadero
Processing Record 4 of Set 1 | tiksi*http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=7735c52e8cd85cbee274b995f12a83a3&q=tiksi
Processing Record 5 of Set 1 | rikitea*http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=7735c52e8cd85cbee274b995f12a83a3&q=rikitea
Processing Record 6 of Set 1 | hilo*http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=7735c52e8cd85cbee274b995f12a83a3&q=hilo
Processing Record 7 of Set 1 | bluff*http://ap

In [49]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)


,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Khani,GE,41.96,42.96,26.17,78,100,1.77,Clouds
1,Qaanaaq,GL,77.48,-69.36,-7.44,98,100,0.49,Clouds
2,Desaguadero,PE,-16.57,-69.04,47.17,76,100,8.10,Clouds
3,Tiksi,RU,71.69,128.87,-16.80,99,98,1.36,Clouds
4,Rikitea,PF,-23.12,-134.97,80.29,71,100,14.36,Clouds
5,Hilo,US,19.73,-155.09,78.53,72,100,12.66,Clouds
6,Bluff,NZ,-46.60,168.33,65.80,73,46,6.26,Clouds
7,Itoman,JP,26.12,127.67,75.58,75,75,4.61,Clouds
8,Sao Filipe,CV,14.90,-24.50,71.17,72,0,15.79,Clear
9,Bolitoc,PH,15.74,119.88,87.15,60,54,4.92,Clouds


In [50]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")